This notebook is a quick cleaning of the data just to get it usable for our baseline ML models. This is quick and dirty cleaning with further refinements (feature eng, cat encoding, etc) to be done later. Different models will require different cleaning so this mostly just combines all the seperate csv files and establishes train-val, and holdout_test set

In [1]:
import os, warnings, gc, requests, json, re
from requests.auth import HTTPDigestAuth
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)

In [2]:
data_path = 'data/' # replace with name of your data file if different
key = '' # put your api key here
acc_id = '' # put your account id here

The cleaning function does some very simple cleaning like converting interest rates from strings to floats.

In [3]:
def cleaning(df):
            
    # drop non_loan columns
    index = df[df['loan_amnt'].isnull()].index
    df.drop(index=index, inplace=True)
    
    # convert % rate from str to float
    df['int_rate'] = df['int_rate'].str[:-1].astype(float)
    df['revol_util'] = df['revol_util'].str[:-1].astype(float)
    
    # convert term from str to int
    df['term'] = df['term'].str.strip().str[:2].astype(int)
    
    # drop weird old columns
    index = (df[(df['loan_status'] == 'Does not meet the credit policy. Status:Charged Off')
            |(df['loan_status'] == 'Does not meet the credit policy. Status:Fully Paid')]).index
    df.drop(index=index, inplace=True)
    
    # convert dates to useable formats
    df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y')
    df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')
    
    # convert earliest cr_line from date to months since loan request
    df['earliest_cr_line'] = ((df['issue_d'] - df['earliest_cr_line']) / np.timedelta64(1, 'M')).astype(int)
    
    return df 

The historical data is split into different csv files. The get data function will clean each one, then combine them all into one dataframe.

In [4]:
def get_data(approved_files, data_path):
    df = pd.DataFrame()
    for file in approved_files:
        print(f'reading in {file}')
        temp_df = pd.read_csv(data_path+file,header=1)
        temp_df = cleaning(temp_df)
        df = pd.concat([df,temp_df],ignore_index=True)
    del temp_df
    return df

My data folder also contains info on rejected loans. Bc of this, make a list of files containing only the csv files containing approved and issued loans

In [5]:
files = os.listdir(data_path)
# only files with approved loans start with 'L'
approved_files = [f for f in files if f[0]=='L']

Clean and combine all historical loan files into one.

In [6]:
df = get_data(approved_files, data_path)

reading in LoanStats_securev1_2018Q4.csv
reading in LoanStats3b_securev1.csv
reading in LoanStats3c_securev1.csv
reading in LoanStats3d_securev1.csv
reading in LoanStats_securev1_2018Q2.csv
reading in LoanStats_securev1_2018Q3.csv
reading in LoanStats_securev1_2018Q1.csv
reading in LoanStats_securev1_2019Q1.csv
reading in LoanStats_securev1_2019Q2.csv
reading in LoanStats_securev1_2019Q3.csv
reading in LoanStats_securev1_2017Q1.csv
reading in LoanStats_securev1_2017Q2.csv
reading in LoanStats_securev1_2017Q3.csv
reading in LoanStats_securev1_2017Q4.csv
reading in LoanStats_securev1_2016Q2.csv
reading in LoanStats3a_securev1.csv
reading in LoanStats_securev1_2016Q3.csv
reading in LoanStats_securev1_2016Q1.csv
reading in LoanStats_securev1_2016Q4.csv


In [7]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,...,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,145073496,NaN,6000.0,6000.0,6000.0,36,14.47,206.44,C,C2,Manager,10+ years,RENT,55000.0,Verified,2018-12-01,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,104xx,NY,10.65,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Oct-2019,ACTIVE,Oct-2019,3479.0,60.01,24.0
1,145548792,NaN,24000.0,24000.0,24000.0,36,10.33,778.14,B,B1,Maintenance Manager,10+ years,MORTGAGE,100000.0,Not Verified,2018-12-01,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,606xx,IL,19.14,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,145565309,NaN,30000.0,30000.0,30000.0,36,15.02,1040.26,C,C3,Firefighter,10+ years,RENT,90000.0,Verified,2018-12-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,681xx,NE,22.23,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,145614543,NaN,8000.0,8000.0,8000.0,36,12.98,269.48,B,B5,Controller,4 years,MORTGAGE,95000.0,Source Verified,2018-12-01,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,480xx,MI,9.44,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,145619975,NaN,22400.0,22400.0,22400.0,60,16.14,546.40,C,C4,Paramedic/ Fire Fighter,10+ years,OWN,56000.0,Source Verified,2018-12-01,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,234xx,VA,18.47,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.shape

(2647537, 150)

In [9]:
# to view all column names
#list(df.columns)

The dataframe currently holds information not available at the time of investment.

Enter your LC account info to preview the currently listed loans. From the currently available loans create a list of all available features. These are the features we know we can use in our ML models. Then try to match these features to the ones available in the Lending Club historical data.

In [10]:
api_key =  {'Authorization': key} 
acc_id = acc_id 

# get loan listings data
loans = 'https://api.lendingclub.com/api/investor/v1/loans/listing'
res = requests.get(loans, headers=api_key)
data = json.loads(res.text)

# grabs the available features
avail_cols = list(data['loans'][0].keys())

In [11]:
# to view columns from the listings api
#avail_cols

Feature names from the api differ from the historical data. Since it's mostly slight formatting differences regex functions will be used to match column names. A dictionary is created to map other feature names from the api to the historical data. Some of the features mapped in the dictionary could also be matched using regex

In [12]:
to_map = {'secAppCollections12MthsExMed': 'sec_app_collections_12_mths_ex_med',
          'secAppInqLast6Mths': 'sec_app_inq_last_6mths',
          'numAcctsEver120Ppd': 'num_accts_ever_120_pd',
          'inqLast6Mths': 'inq_last_6mths',
          'numTl120dpd2m': 'num_tl_120dpd_2m',
          'numTl30dpd': 'num_tl_30dpd',
          'numTl90gDpd24m': 'num_tl_90g_dpd_24m',
          'numTlOpPast12m': 'num_tl_op_past_12m',
          'collections12MthsExMed': 'collections_12_mths_ex_med',
          'isIncV': 'verification_status',
          'isIncVJoint': 'verification_status_joint',
          'openIl12m': 'open_il_12m',
          'openIl24m': 'open_il_24m',
          'openRv12m': 'open_rv_12m',
          'openRv24m': 'open_rv_24m',
          'secAppChargeoffWithin12Mths': 'sec_app_chargeoff_within_12_mths',
          'addrZip': 'zip_code',
          'accOpenPast24Mths': 'acc_open_past_24mths',
          'chargeoffWithin12Mths': 'chargeoff_within_12_mths',
          'inqLast12m': 'inq_last_12m',
          'delinq2Yrs': 'delinq_2yrs',
          'percentBcGt75': 'percent_bc_gt_75',
          'loanAmount': 'loan_amnt',
          'iLUtil': 'il_util',          
         }

# cols w info regarding loan performace from LC dataset
# this will get stored with the data used for modeling so we can see returns on investment later
cols_of_interest = ['issue_d','loan_status','total_pymnt', 'total_rec_int',
                    'total_rec_late_fee','total_rec_prncp', 'recoveries',
                    'collection_recovery_fee', 'last_pymnt_d']
                    

# cols dropped from the listed loans features (these features are not in historical data)
to_drop = ['reviewStatus', 'housingPayment', 'creditPullD', 'ilsExpD', 'mtgPayment', 'expD', 'acceptD',
          'investorCount','serviceFeeRate', 'disbursementMethod', 'listD', 'expDefaultRate',
          'reviewStatusD','fundedAmount']

The next block calls the lendingclub api to get a list of available loans. From this we get the available features to use for our models. You can use your api key and investor id.

In [13]:
# performs feature matching between features from api call and features from dataset
# some features were easy to match with regex but for others is was quick to manually write mapping
# this can probably be clean up further
api_cols = []
for col in avail_cols:
    if col in list(to_map.keys()):
        new_col = to_map[col]
        api_cols.append(new_col)
        continue
    if col in to_drop:
        continue
    new_col = re.sub(r'([A-Z])', r'_\1', col).lower()
    new_col = re.sub(r'([0-9])+', r'_\1', new_col).lower()
    api_cols.append(new_col)

In [14]:
api_cols[:10]

['id',
 'member_id',
 'loan_amnt',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_length',
 'home_ownership']

In [15]:
len(api_cols)

105

api_cols is a list of usable features from the api call listed the feature names in the historical data. Now reassign the dataframe into a df with only the features we want.

In [16]:
df = df[api_cols+cols_of_interest]

In [17]:
df.head()

,id,member_id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,desc,purpose,zip_code,addr_state,initial_list_status,emp_title,acc_now_delinq,acc_open_past_24mths,bc_open_to_buy,percent_bc_gt_75,bc_util,dti,delinq_2yrs,...,total_cu_tl,inq_last_12m,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,revol_bal_joint,open_act_il,sec_app_open_act_il,issue_d,loan_status,total_pymnt,total_rec_int,total_rec_late_fee,total_rec_prncp,recoveries,collection_recovery_fee,last_pymnt_d
0,145073496,NaN,6000.0,36,14.47,206.44,C,C2,10+ years,RENT,55000.0,Verified,NaN,credit_card,104xx,NY,w,Manager,0.0,2.0,344.0,100.0,92.0,10.65,0.0,...,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2018-12-01,Charged Off,1106.520000,272.38,0.0,546.14,288.0,51.84,Apr-2019
1,145548792,NaN,24000.0,36,10.33,778.14,B,B1,10+ years,MORTGAGE,100000.0,Not Verified,NaN,debt_consolidation,606xx,IL,w,Maintenance Manager,0.0,9.0,17353.0,16.7,33.0,19.14,0.0,...,0.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,2018-12-01,Current,9317.020000,2124.32,0.0,7192.70,0.0,0.00,Dec-2019
2,145565309,NaN,30000.0,36,15.02,1040.26,C,C3,10+ years,RENT,90000.0,Verified,NaN,debt_consolidation,681xx,NE,w,Firefighter,0.0,5.0,22030.0,50.0,47.0,22.23,0.0,...,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2018-12-01,Fully Paid,30883.673712,883.67,0.0,30000.00,0.0,0.00,Feb-2019
3,145614543,NaN,8000.0,36,12.98,269.48,B,B5,4 years,MORTGAGE,95000.0,Source Verified,NaN,credit_card,480xx,MI,w,Controller,0.0,10.0,22866.0,20.0,33.3,9.44,0.0,...,0.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2018-12-01,Current,6855.630000,562.97,0.0,6292.66,0.0,0.00,Dec-2019
4,145619975,NaN,22400.0,60,16.14,546.40,C,C4,10+ years,OWN,56000.0,Source Verified,NaN,credit_card,234xx,VA,w,Paramedic/ Fire Fighter,0.0,2.0,15054.0,25.0,46.6,18.47,0.0,...,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,2018-12-01,Current,6526.670000,3357.58,0.0,3169.09,0.0,0.00,Dec-2019


In [18]:
# save data
df.to_pickle('data/prelim-all.pkl')

Next create train/val/test splits. Since only historical data will be avaible when choosing which loans to invest in, I chose to split the data by date. I only use 36m term loans instead of both 36 and 60m because I do not want my money locked in an investment for 5 years. 

Most recent data is Q3 2019 so 36m loans from Q3 2016 and earlier should be fully paid off or confirmed default by now. This most recent loan data from Q4 2015- Q3 2016 will be used as the test set.

Train and Validation sets will have loans from Q3 2015 and earlier

In [19]:
df = pd.read_pickle('data/prelim-all.pkl')

Chose to restric to 36 month loans because so we have full payment info on more recent data. The other loan term is 5 years and would limit the time periods available for a train/val/holdout data split.

In [20]:
# restrict to 36m loans
df = df[df['term']==36]

In [21]:
df.shape

(1871233, 114)

Next a few features are added to later calculate annualized returns.

In [22]:
# how long until the loan was paid off/reached default status
df['loan_length'] = (pd.to_datetime(df['last_pymnt_d'])-df['issue_d']).dt.days

# loans that did not recieve any payments had NA values
# by adding 30 days it provides a timeframe to calculate returns
# NA values messed up the calculation
df['loan_length'] = df['loan_length'].fillna(30)

# loans that were paid off in the same month had a loan length of 0
# loan length of zero pushed returns to infinity
# loans also take some time to originate after initial investment
# adding 30 days creates cleaner and more realistic return calculations
df.loc[df['loan_length']==0, 'loan_length'] = 30

# recoveries are not factored into the last_pyment_d feature
# without this feature a loan can make the first payment, then default and make partial recovry pymnt
# and return on investment would be above 100%
# adding a year for the recovery peroid is also fairly realistic
df.loc[df['recoveries']>0, 'loan_length'] =df.loc[df['recoveries']>0, 'loan_length']+365

# standard annualized return formula
df['returns'] = (df['total_pymnt']/df['loan_amnt'])**(365/(df['loan_length'])) - 1


In [23]:
df.sort_values('returns').head()

,id,member_id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,desc,purpose,zip_code,addr_state,initial_list_status,emp_title,acc_now_delinq,acc_open_past_24mths,bc_open_to_buy,percent_bc_gt_75,bc_util,dti,delinq_2yrs,...,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,revol_bal_joint,open_act_il,sec_app_open_act_il,issue_d,loan_status,total_pymnt,total_rec_int,total_rec_late_fee,total_rec_prncp,recoveries,collection_recovery_fee,last_pymnt_d,loan_length,returns
1597052,159096586,NaN,10000.0,36,8.81,317.12,A,A5,2 years,RENT,46000.0,Not Verified,NaN,credit_card,752xx,TX,w,Branch Manager,0.0,6.0,15528.0,0.0,38.4,21.84,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,2019-09-01,Current,312.23,85.66,0.0,226.57,0.0,0.0,Oct-2019,30.0,-1.0
1382894,148033283,NaN,7000.0,36,6.46,214.42,A,A1,< 1 year,MORTGAGE,205987.0,Not Verified,NaN,debt_consolidation,834xx,ID,w,Public Awareness Coordinator,0.0,1.0,13852.0,33.3,43.9,5.15,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2019-02-01,Charged Off,0.00,0.00,0.0,0.00,0.0,0.0,NaN,30.0,-1.0
1647181,156170616,NaN,18900.0,36,25.00,751.47,D,D4,9 years,RENT,45000.0,Verified,NaN,debt_consolidation,010xx,MA,w,Server,0.0,1.0,2659.0,50.0,86.4,21.25,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2019-08-01,Late (31-120 days),0.00,0.00,0.0,0.00,0.0,0.0,NaN,30.0,-1.0
1587217,159587785,NaN,6000.0,36,12.40,200.44,B,B4,10+ years,MORTGAGE,60000.0,Source Verified,NaN,credit_card,432xx,OH,w,Computer Operator,0.0,7.0,13223.0,50.0,71.1,36.29,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2019-09-01,Current,196.31,57.87,0.0,138.44,0.0,0.0,Oct-2019,30.0,-1.0
1475127,153519765,NaN,7000.0,36,10.33,226.96,B,B1,NaN,OWN,52000.0,Not Verified,NaN,debt_consolidation,283xx,NC,w,NaN,0.0,6.0,6802.0,0.0,35.8,16.96,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2019-06-01,Charged Off,222.94,30.13,0.0,192.81,0.0,0.0,Jul-2019,30.0,-1.0


In [24]:
df.sort_values('returns').tail()

,id,member_id,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,desc,purpose,zip_code,addr_state,initial_list_status,emp_title,acc_now_delinq,acc_open_past_24mths,bc_open_to_buy,percent_bc_gt_75,bc_util,dti,delinq_2yrs,...,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,revol_bal_joint,open_act_il,sec_app_open_act_il,issue_d,loan_status,total_pymnt,total_rec_int,total_rec_late_fee,total_rec_prncp,recoveries,collection_recovery_fee,last_pymnt_d,loan_length,returns
2016565,113730750,NaN,35000.0,36,30.79,1501.00,G,G1,5 years,RENT,100000.0,Verified,NaN,debt_consolidation,551xx,MN,w,Owner / President,0.0,6.0,3601.0,50.0,57.6,18.83,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2017-08-01,Fully Paid,36837.386580,1837.39,0.0,35000.0,0.00,0.0,Sep-2017,31.0,0.826550
1397041,147450393,NaN,5000.0,36,27.27,204.86,E,E5,3 years,RENT,40500.0,Source Verified,NaN,debt_consolidation,853xx,AZ,w,Heavy Equipment Operator,0.0,1.0,169.0,50.0,88.7,21.45,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2019-02-01,Fully Paid,5248.242190,248.24,0.0,5000.0,0.00,0.0,Mar-2019,28.0,0.880710
1400045,147457038,NaN,15000.0,36,25.34,599.10,E,E3,< 1 year,RENT,70000.0,Not Verified,NaN,debt_consolidation,951xx,CA,w,Consumer Loan Underwriter,0.0,2.0,0.0,100.0,101.2,22.90,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2019-02-01,Fully Paid,15754.433017,754.43,0.0,15000.0,0.00,0.0,Mar-2019,28.0,0.895880
1956953,118567343,NaN,4200.0,36,29.69,177.59,F,F2,NaN,RENT,32496.0,Source Verified,NaN,other,852xx,AZ,f,NaN,0.0,7.0,5657.0,0.0,21.4,7.98,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2017-09-01,Fully Paid,4430.433854,230.43,0.0,4200.0,0.00,0.0,Oct-2017,30.0,0.915269
2645214,90835542,NaN,10800.0,36,19.99,401.32,D,D4,6 years,OWN,54559.0,Verified,NaN,home_improvement,324xx,FL,w,Sraff,0.0,1.0,3000.0,0.0,0.0,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2016-10-01,Charged Off,23439.240000,0.00,0.0,0.0,23439.24,0.0,NaN,395.0,1.046262


The train/val/holdout split is designed to simulate future investments.
Train and validate on historical data, and the (test)holdout set is the most recent loan data with full payment/chargoff information (3 years ago).

In [25]:
# split train and test data
test_end = pd.datetime(2016,9,1)
test_start = pd.datetime(2015,10,1)
test_df = df[(df['issue_d'] >= test_start) & (df['issue_d'] <= test_end)]

In [26]:
# verify start and end dates for test data
test_df.issue_d.min(), test_df.issue_d.max()

(Timestamp('2015-10-01 00:00:00'), Timestamp('2016-09-01 00:00:00'))

In [27]:
#export
train_end = pd.datetime(2015,9,1)
train_df = df[df['issue_d'] <= train_end]

In [28]:
# verify start and end dates for test data
train_df.issue_d.min(), train_df.issue_d.max()

(Timestamp('2007-06-01 00:00:00'), Timestamp('2015-09-01 00:00:00'))

Verify no test loans leak into train data. Testing intersection between sets much faster than finding overlap between lists of ids.

In [29]:
train_ids = set(train_df.id)
test_ids = set(test_df.id)

In [30]:
train_ids.intersection(test_ids)

set()

Check loan status within each data split.

In [31]:
train_df['loan_status'].value_counts()

Fully Paid     457060
Charged Off     72813
Current             2
Name: loan_status, dtype: int64

In [32]:
train_df = train_df[(train_df['loan_status']=='Fully Paid')|(train_df['loan_status']=='Charged Off')]

In [33]:
train_df['loan_status'].value_counts()

Fully Paid     457060
Charged Off     72813
Name: loan_status, dtype: int64

In [34]:
test_df['loan_status'].value_counts()

Fully Paid            280747
Charged Off            52415
Current                   77
Late (31-120 days)        77
In Grace Period           11
Late (16-30 days)          5
Default                    2
Name: loan_status, dtype: int64

In [35]:
# for simplicity
test_df = test_df[(test_df['loan_status']=='Fully Paid')|(test_df['loan_status']=='Charged Off')]

In [36]:
test_df['loan_status'].value_counts()

Fully Paid     280747
Charged Off     52415
Name: loan_status, dtype: int64

In [37]:
train_df['application_type'].value_counts()

Individual    529873
Name: application_type, dtype: int64

In [38]:
test_df['application_type'].value_counts()

Individual    330206
Joint App       2956
Name: application_type, dtype: int64

In [39]:
test_df = test_df[test_df['application_type']!='Joint App']

In [40]:
test_df['application_type'].value_counts()

Individual    330206
Name: application_type, dtype: int64

In [41]:
train_df.to_pickle('data/api_train_df.pkl')
test_df.to_pickle('data/api_test_df.pkl')